In [ ]:
!pip install transformers 
# !pip install torch
# !pip install ipypublish

     |████████████████████████████████| 2.3MB 16.0MB/s 
     |████████████████████████████████| 3.3MB 60.7MB/s 
     |████████████████████████████████| 901kB 56.8MB/s 


In [ ]:
# from ipypublish import nb_setup
import os
import time
import datetime
from google.colab import drive
import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt
% matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load Preprocessed Data

In [ ]:
import re
review = {}
review['preprocessed']=pd.read_csv("/content/drive/My Drive/NLP/review_with_scores.csv")
review['preprocessed']=review['preprocessed'].sort_values("automated_readability_index",ascending=False)
review['preprocessed']=review['preprocessed']["text"].reset_index(drop=True)


In [ ]:
#set the sample_size and split the train and validate data

review_sample_size = 20000
review['final'] = review['preprocessed']
review['sample'] = review['final'].reset_index(drop=True)[:review_sample_size]
review['validate'] = review['final'].reset_index(drop=True)[review_sample_size:review_sample_size+int(review_sample_size/10)]

#Global Setting

In [ ]:
# Set the seed 
random_state = 42

random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state) #seed for generating random numbers for the current GPU
# Set the batch size before train the data
batch_size = 2  #The batch size defines the number of samples that will be propagated through the network.

# Loading GPT2 Confconfiguration, tokenizer and Pretrained Model 

In [ ]:
# GPT2 Confconfiguration
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
# GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') 
#  Pretrained Model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# set pytorch to run on the GPU.
device = torch.device("cuda")
model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
#tokenize the data, which is the process of converting a sequence of characters into tokens

class GPT2_dataset(Dataset):

  def __init__(self, text_list, tokenizer, gpt2_type="gpt2", max_length=768): #GPT2-small
  #The maximum length (in number of tokens) for the inputs to the transformer model.

    self.tokenizer = tokenizer
    self.input_reviews = []
    self.attn_masks = []

    for txt in text_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
      self.input_reviews.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_reviews)

  def __getitem__(self, index):
    return self.input_reviews[index], self.attn_masks[index] 

In [ ]:
#Covert Data to GPT2 required format
dataset = GPT2_dataset(review['sample'], tokenizer, max_length=768)
val_dataset = GPT2_dataset(review['validate'], tokenizer, max_length=768)

In [ ]:
#for pytorch framework
train_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler(dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = RandomSampler(val_dataset), # Pull out batches randomly.
            batch_size = batch_size # Evaluate with this batch size.
        )

# Finetune GPT2 Language Model

In [ ]:
# Pytorch Defalt Parameter Setting
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

In [ ]:
#Define optimizer
optimizer = AdamW(model.parameters(),lr = learning_rate,eps = epsilon)

In [ ]:
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps ,num_training_steps = total_steps)

In [ ]:
#time format
def format_time(elapsed):
  time=datetime.timedelta(seconds=int(round((elapsed))))
  return str(time)

In [ ]:
#Train model
start_time = time.time()
model = model.to(device) #cuda -->GPU

for i in range(0, epochs):
  
    print("")
    print('======== Epoch {:} / {:} ========'.format(i + 1, epochs))

    epoch_start_time = time.time()
    total_train_loss = 0
    model.train() #start to train

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        
        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None)
        loss = outputs[0].item() 
        total_train_loss += loss
        
        outputs[0].backward() #loss backward
        optimizer.step()
        scheduler.step()

    # Overall average loss 
    avg_train_loss = total_train_loss / len(train_dataloader)         
    # Each epoch running time
    training_time = format_time(time.time() - epoch_start_time)

    print("")
    print("  Average training loss: ", round(avg_train_loss,4))
    print("  Epoch running time:",training_time)
print("")
print("Complete Training")
print("Total running time ",(format_time(time.time()-start_time)))


======== Epoch 1 / 5 ========

  Average training loss:  0.3274
  Epoch running time: 0:48:00

======== Epoch 2 / 5 ========

  Average training loss:  0.2714
  Epoch running time: 0:48:01

======== Epoch 3 / 5 ========

  Average training loss:  0.2399
  Epoch running time: 0:48:01

======== Epoch 4 / 5 ========

  Average training loss:  0.1966
  Epoch running time: 0:48:01

======== Epoch 5 / 5 ========

  Average training loss:  0.1507
  Epoch running time: 0:48:01

Complete Training
Total running time  4:00:03


In [ ]:
# Model Evaluation
print("")
print("Running Validation...")

eval_start_time = time.time()
model.eval()

total_eval_loss = 0
nb_eval_steps = 0

# Evaluation for one epoch
for batch in validation_dataloader:
    
    b_input_ids = batch[0].to(device)
    b_labels = batch[0].to(device)
    b_masks = batch[1].to(device)
    
    with torch.no_grad():        
        outputs  = model(b_input_ids, attention_mask = b_masks,labels=b_labels) 
        loss = outputs[0]
                                
    batch_loss = loss.item()
    total_eval_loss += batch_loss        

avg_val_loss = total_eval_loss / len(validation_dataloader)
validation_time = format_time(time.time() - eval_start_time)    

print("  Validation Loss: ",round(avg_val_loss,4))
print("  Validation took: ",validation_time)


Running Validation...
  Validation Loss:  0.3341
  Validation took:  0:01:34


# Model Information

In [ ]:
# Get all of the model's parameters as a list of tuples.

params = list(model.named_parameters())
print('The GPT-2 model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50259, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072

# Save Model

In [ ]:
output_dir = "/content/drive/My Drive/NLP/transformer_model_20000/"
print("Save to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model 
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Save to /content/drive/My Drive/NLP/transformer_model_20000/


('/content/drive/My Drive/NLP/transformer_model_20000/tokenizer_config.json',
 '/content/drive/My Drive/NLP/transformer_model_20000/special_tokens_map.json',
 '/content/drive/My Drive/NLP/transformer_model_20000/vocab.json',
 '/content/drive/My Drive/NLP/transformer_model_20000/merges.txt',
 '/content/drive/My Drive/NLP/transformer_model_20000/added_tokens.json')

## Load Saved Model  

In [ ]:
# output_dir = "/content/drive/My Drive/NLP/transformer_model_save_5000/"
# model = GPT2LMHeadModel.from_pretrained(output_dir)
# tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# Generate Text

In [ ]:
# pip install spacy_readability

In [ ]:
import spacy
from spacy_readability import Readability

def generate_review(num_review=5,len_review=300):
  nlp = spacy.load('en')
  read = Readability()
  nlp.add_pipe(read, last=True)
  model.eval()
  prompt = "<|startoftext|>"
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)
  # print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50,  # add uncertainty to the output
                                  max_length = len_review,
                                  top_p=0.95, # top probability result
                                  num_return_sequences=num_review
                                  )
  print(" ")
  print("Generating the reviews for you.....")
  print(" ")
  for i, sample_output in enumerate(sample_outputs):
    sentence=tokenizer.decode(sample_output, skip_special_tokens=True)
    readability_score = nlp(sentence)._.automated_readability_index
    print("Review ","{}: {}\n".format(i+1, sentence))
    print("Review ","{} Readability Score: {}\n\n".format(i+1, round(readability_score,3)))

In [ ]:
generate_review()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 
Generating the reviews for you.....
 
Review  1: The best and authentic thai food I have had in ages! The owners are very friendly and helpful as well! The food comes out warm and fresh at reasonable prices and the staff are warm and willing to share any additional condiments needed. The interior is clean and comfortable and the prices are reasonable considering the quality of all the dishes we ordered. All the ingredients were amazing and the restaurant is beautiful.

Review  1 Readability Score: 8.194


Review  2: We returned for lunch yesterday during covid 19 phase 1 reopening with friends. We masked and physically distanced with plenty of spacing. Our service was friendly, as usual with the menu items. As always, the food was delicious and plentiful, and the tapas style was great. Special shout out to our masked server lisa. Very personable, very kind and understanding. Special shout out to our masked server lisa!

Review  2 Readability Score: 6.423


Review  3: We just walked i